In [7]:
import glob
import os
from PIL import Image
import numpy as np
import pandas as pd
import scipy.signal

# Image processing tools
import skimage.feature
import skimage.filters
import skimage.filters.rank
import skimage.io
import skimage.morphology
import skimage.segmentation
from scipy import ndimage as ndi

import bokeh
from bokeh.models.widgets import Panel, Tabs
from bokeh.io import output_file, show
from bokeh.plotting import figure

bokeh.io.output_notebook()


import img_seg_package.single_image as si

Loading BokehJS ...

In [8]:
# The overall directory for the examined images
data_dir = '../Lung_Paper_Images/Test_Images/'

# The tissue examined for filename purposes
tissue = 'Lung'

# The variant being examined
variant = 'CAPA4'

# The animal being examined (integer for dataframe purposes)
animal = 2

# The replicate being examined (integer for dataframe purposes)
replicate = 2

# The name of the image being examined
img_loc = '1_XY04_00065_Overlay'

# The complete filename of the image being examined 
fname = data_dir + tissue + '/' + variant + '/' + str(animal) + '/' + str(replicate) + '/' + img_loc + '.tif'

# The dataframe containing the parameters for examination
df_loc = '../Lung_Paper_Images/Quantification/Lung_Quantification.csv'

df = pd.read_csv(df_loc)

df.head()

,Date,Tissue,Size Threshold,Gaussian Size,Truncation,Intensity Threshold,Minimum Size,Virus,Animal,Replicate,Count,Cells Quantified,Brightness List,Applied Threshold,Image Multiplication Factor,Minimum Pixel Value,Maximum Pixel Value,Area Threshold,Total Area
0,6/30/2020,Lung,0.62,5,2,1000,10,AAV5,1,1,84,[ 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 ...,"{'1': 0.09374644248769036, '2': 0.396884378149...",0.301449,0.706774,-0.184533,0.527600,0.020282,31273793.26
1,6/30/2020,Lung,0.62,5,2,1000,10,AAV5,1,2,105,[ 1 2 3 4 5 6 7 8 9 10 11 ...,"{'1': 0.34954984628418456, '2': 0.301035344397...",0.322450,0.699570,-0.204931,0.543257,0.019231,32916921.38
2,6/30/2020,Lung,0.62,5,2,1000,10,AAV5,2,1,101,[ 1 2 3 4 5 6 7 8 9 10 11 ...,"{'1': 0.040592697357633566, '2': 0.07469444680...",0.243228,0.591669,-0.151784,0.571073,0.020107,48020362.14
3,6/30/2020,Lung,0.62,5,2,1000,10,AAV5,2,2,108,[ 1 2 3 4 5 6 7 8 9 10 11 ...,"{'1': 0.08277810712698083, '2': 0.062237664806...",0.396656,0.584836,-0.267212,0.474624,0.019407,42443016.38
4,6/30/2020,Lung,0.62,5,2,1000,10,AAV5,3,1,28,[ 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 ...,"{'1': 0.07123424141933186, '2': 0.198157759863...",0.411377,0.589563,-0.211362,0.348148,0.019425,26451775.48


In [1]:
inds = (df['Virus'] == variant) & (df['Animal'] == animal) & (df['Replicate'] == replicate)
    
im_af = skimage.img_as_float(skimage.io.imread(fname)[:,:,0])
im_sig = skimage.img_as_float(skimage.io.imread(fname)[:,:,1])

#Save the intermediate images for the figure
#skimage.io.imsave()

NameError: name 'df' is not defined

In [9]:


im_sub = im_sig - im_af*df.loc[inds, 'Image Multiplication Factor'].values[0]

im_height = np.shape(im_sub)[0]
im_width = np.shape(im_sub)[1]

template = np.load('../template.npy')

template_height = np.shape(template)[0]
template_width = np.shape(template)[1]

threshed_single = im_af > df.loc[inds, 'Area Threshold'].values[0]

im_bg = skimage.filters.gaussian(im_sub, df.loc[inds, 'Gaussian Size'].values[0], truncate = df.loc[inds, 'Truncation'].values[0])

im_no_bg = im_sub - im_bg

im_no_bg = (im_no_bg - df.loc[inds, 'Minimum Pixel Value'].values[0]) / (df.loc[inds, 'Maximum Pixel Value'].values[0] - df.loc[inds, 'Minimum Pixel Value'].values[0])

im_feat = skimage.feature.match_template(im_sub, template)

binary_thresh = im_feat > df.loc[inds, 'Size Threshold'].values[0]

binary_gauss = im_no_bg[int(template_height/2 - 1):int(im_height - int(template_height/2)), int(template_width/2 - 1):int(im_width - int(template_width/2))] > df.loc[inds, 'Applied Threshold'].values[0]

binary_mult = binary_gauss*binary_thresh

binary_rem = skimage.morphology.remove_small_objects(binary_mult, min_size=df.loc[inds, 'Minimum Size'].values[0])

im_labeled, n_labels = skimage.measure.label(binary_rem, background=0, return_num=True)

# Load phase image
im_p = skimage.img_as_float(skimage.io.imread(fname))[:,:,:2][int(template_height/2 - 1):int(im_height - int(template_height/2)), int(template_width/2 - 1):int(im_width - int(template_width/2))]

binary_rgb = np.dstack((binary_rem, binary_rem, binary_rem))